In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

import os, sys, argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from glob import glob
from PIL import Image
from copy import copy

os.environ['CUDA_VISIBLE_DEVICES'] = '7'

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from siuba import *
from plotnine import *

In [2]:
sys.path.append('model_opts')
from feature_extraction import *
from feature_reduction import *
from model_options import *
from mapping_methods import *

from processing import *

In [3]:
image_transforms = get_recommended_transforms('taskonomy')

In [4]:
imageset = 'oasis'
asset_dictlist = []
root = 'images/{}/'.format(imageset)
assets = glob(root + '*.jpg')
for asset in assets:
    imgstr = asset.split('/')[-1]
    row = {'image_name': imgstr,
           'dataset': imageset}
    asset_dictlist.append(row)
image_df = (pd.DataFrame(asset_dictlist)
            .sort_values(by='image_name', ignore_index=True))

In [5]:
class StimulusSet(Dataset):
    def __init__(self, csv, image_transforms=None):
        
        self.transforms = image_transforms
        
        if isinstance(csv, pd.DataFrame):
            self.df = csv
        if isinstance(csv, str):
            self.df = pd.read_csv(csv)
        
        self.roots = self.df.dataset
        self.images = self.df.image_name

    def __getitem__(self, index):
        filename = os.path.join('images',self.roots.iloc[index], self.images.iloc[index])
        img = Image.open(filename).convert('RGB')
        
        if self.transforms:
            img = self.transforms(img)
            
        return img
    
    def __len__(self):
        return len(self.images)

In [6]:
stimulus_loader = DataLoader(dataset=StimulusSet(image_df, image_transforms), batch_size=64)

In [7]:
response_data = load_response_data('oasis', 'response')
response_data = copy(image_df).merge(response_data, on = 'image_name')

### Effective Dimensionality Method

In [8]:
def calculate_effective_dims(x):
    pca = PCA().fit(x)
    eigvals = np.array(sorted(pca.explained_variance_))
    
    K = len(eigvals)
    eigen_sum = sum(eigvals)
    norm_eigvals = eigvals/eigen_sum
    eigen_var = np.var(eigvals)*((K-1)/K)
    
    return np.prod(norm_eigvals**(-norm_eigvals))

### Taskonomy Effective Dims

In [9]:
reg_redux = pd.read_csv('results/neural_reg_redux.csv') >> filter(_.score_type == 'pearson_r')

In [10]:
reg_redux_layers = (reg_redux.groupby(['model','train_type','model_layer'])['score'].mean().reset_index())

In [11]:
target_layers = (reg_redux_layers >> group_by(_.model, _.train_type) >> filter(_.score == np.max(_.score)) >>
                 filter(_.train_type == 'taskonomy') >> ungroup()).set_index('model').to_dict(orient='index')

In [16]:
effective_dimlist = []

model_options = get_model_options(train_type = 'taskonomy')
for model_name in tqdm(target_layers):
    model_string = model_name + '_taskonomy'
    target_layer = target_layers[model_name]['model_layer']

    model_name = model_options[model_string]['model_name']
    train_type = model_options[model_string]['train_type']
    model_call = model_options[model_string]['call']

    model = eval(model_call)
    model = model.eval()
    if torch.cuda.is_available():
        model = model.cuda()
        
    target_features = get_all_feature_maps(model, stimulus_loader, layers_to_retain = [target_layer])
    
    feature_redux = srp_extraction(model_string, feature_maps = target_features, delete_saved_outputs = False,
                                   output_dir = 'srp_arrays/{}'.format(imageset))
    
    edim = calculate_effective_dims(feature_redux[target_layer])
    
    print('Model: {} | Effective Dimensionality: {}'.format(model_name, edim))
    
    effective_dimlist.append({'model': model_name, 'model_layer': target_layer,
                              'effective_dimensionality': edim})

  0%|          | 0/25 [00:00<?, ?it/s]

Feature Extraction (Batch):   0%|          | 0/15 [00:00<?, ?it/s]

Computing 5830 SRPs for autoencoding_taskonomy on TITAN RTX...


SRP Extraction (Layer):   0%|          | 0/1 [00:00<?, ?it/s]

Model: autoencoding | Effective Dimensionality: 196.40091197019618


...

Model: segment_unsup2d | Effective Dimensionality: 399.8786710796452


Feature Extraction (Batch):   0%|          | 0/15 [00:00<?, ?it/s]

Computing 5830 SRPs for vanishing_point_taskonomy on TITAN RTX...


SRP Extraction (Layer):   0%|          | 0/1 [00:00<?, ?it/s]

Model: vanishing_point | Effective Dimensionality: 469.22314351949905


In [17]:
task_effect_dims = pd.DataFrame(effective_dimlist)

In [18]:
task_effect_dims.sort_values(by='effective_dimensionality')

,model,model_layer,effective_dimensionality
11,inpainting,ReLU-41,11.506803
13,keypoints2d,ReLU-50,45.010895
4,denoising,MaxPool2d-2,165.875887
18,random_weights,Conv2d-4,170.243560
0,autoencoding,ReLU-11,196.400912
12,jigsaw,MaxPool2d-1,270.940708
15,nonfixated_pose,ReLU-27,361.229301
17,point_matching,ReLU-12,382.376321
23,segment_unsup2d,ReLU-40,399.878671
10,fixated_pose,ReLU-24,402.512301
